In [418]:
import pandas as pd
import numpy as np

In [419]:
df = pd.read_csv('../csv/time_series_covid19_recovered_global.csv', index_col=0)
df = df.drop(columns=['Lat', 'Long'])


In [420]:
df = df.reset_index().set_index('Country/Region')
df = df.rename({'Congo (Brazzaville)': 'Congo', 'Congo (Kinshasa)': 'Democratic Republic of Congo'})

In [421]:
def merge_provinces(country_name, df):
    country = df.loc[[country_name]].drop(columns=['Province/State'])
    country = country.sum(axis=0).rename(country_name)
    df = df.drop([country_name]).append(country)
    return df

In [422]:
# Dropping the ships
df = df.drop(['Diamond Princess'])
df = df.drop(['MS Zaandam'])
# Merge countries with provinces into a single row
df = merge_provinces('China', df)
df = merge_provinces('Australia', df)

In [423]:
mask = (df['Province/State'] != '0') & (pd.isnull(df['Province/State']) != True)
df2 = df.loc[mask]
df2 = df2.set_index('Province/State')
df2.index = df2.index.rename('Country/Region')

In [424]:
df.loc['China']

Province/State      NaN
1/22/20              28
1/23/20              30
1/24/20              36
1/25/20              39
                  ...  
3/31/20           76206
4/1/20            76405
4/2/20            76565
4/3/20            76760
4/4/20            76946
Name: China, Length: 75, dtype: object

In [425]:
df = df[(df['Province/State'] == '0') | (pd.isnull(df['Province/State']) == True)]

In [426]:
df = df.append(df2)
df = df.drop(columns='Province/State')

In [427]:
df.loc['China']

1/22/20       28.0
1/23/20       30.0
1/24/20       36.0
1/25/20       39.0
1/26/20       49.0
            ...   
3/31/20    76206.0
4/1/20     76405.0
4/2/20     76565.0
4/3/20     76760.0
4/4/20     76946.0
Name: China, Length: 74, dtype: float64

In [428]:
df = df.stack().reset_index(level=1)
df = df.rename(columns={'level_1': 'date', 0: 'recovered'})

In [429]:
df['date'] = pd.to_datetime(df['date']).apply(lambda x: x.strftime('%Y-%m-%d'))

In [430]:
df = df.rename({'Cabo Verde': 'Cape Verde', 'US': 'United States', 'West Bank and Gaza': 'Palestine', 'Burma': 'Myanmar', 'Channel Islands': 'Jersey', 'Holy See': 'Vatican', 'Korea, South': 'South Korea', 'Falkland Islands (Islas Malvinas)': 'Falkland Islands', 'Sint Maarten': 'Sint Maarten (Dutch part)', 'Taiwan*': 'Taiwan'})

In [431]:
full_data = pd.read_csv('../data/full_data.csv', index_col=0)

In [436]:
difference = df.index.difference(full_data.index).to_list()
full_diff = full_data.index.difference(df.index).to_list()

In [437]:
difference

['French Guiana',
 'Guadeloupe',
 'Martinique',
 'Mayotte',
 'Reunion',
 'St Martin']

In [438]:
full_diff

['Guam',
 'Guernsey',
 'International',
 'Northern Mariana Islands',
 'Puerto Rico',
 'United States Virgin Islands',
 'World']